# 차선검출

## <br>라이브러리

In [1]:
import cv2
import numpy as np

## <br>함수부

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255):    # ROI 설정
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
        color = color3
    else:
        color = color1
        
    cv2.fillPoly(mask, vertices, color)    # ROI부분을 color로 채움
    
    ROI_image = cv2.bitwise_and(img, mask)
    return ROI_image

def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):    # 허프변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    
    return line_img

def weighted_img(img, initial_img, a=1, b=1., c=0.):    # 두 이미지 겹치기
    return cv2.addWeighted(initial_img, a, img, b, c)

## <br>이미지 처리 (grayscale - invert - blur - canny)

In [3]:
original_image = cv2.imread('C:/Users/admin/Desktop/Hustar/Project/linedetect/source.jpg')
gray_image = grayscale(original_image)
invert_image = cv2.bitwise_not(gray_image)
blur_image = gaussian_blur(invert_image, 5)
canny_image = canny(blur_image, 70, 100)

original_image_resized = cv2.resize(original_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
gray_image_resized = cv2.resize(gray_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
invert_image_resized = cv2.resize(invert_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
blur_image_resized = cv2.resize(blur_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
canny_image_resized = cv2.resize(canny_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
cv2.imshow('original', original_image_resized)
cv2.imshow('gray', gray_image_resized)
cv2.imshow('invert', invert_image_resized)
cv2.imshow('blur', blur_image_resized)
cv2.imshow('canny', canny_image_resized)

cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>ROI 위치 설정

invert test용 라인 (ROI 위치 확인용)

In [5]:
height, width = original_image.shape[:2]
vertices = np.array([[(width/4-45, height), (width/3+45, height/3+45), (width*2/3-45, height/3+45), (width*3/4+45, height)]], dtype=np.int32)
roi_image = region_of_interest(invert_image, vertices)
cv2.imshow('ROI', roi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

canny 실전용 라인

In [6]:
height, width = original_image.shape[:2]
# 여기에 vertice 라인 넣음. 추후에
roi_image = region_of_interest(canny_image, vertices)
cv2.imshow('ROI', roi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>HoughLine 적용

In [7]:
hough_img = hough_lines(roi_image, 1, 1*np.pi/180, 30, 65, 20)    # 허프 변환
result = weighted_img(hough_img, original_image)
cv2.imshow('result', result)

cv2.waitKey(0)
cv2.destroyAllWindows()